In [13]:
import itertools
import numpy as np
import pandas as pd


# !wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz
!tar xvfz LKH-3.0.7.tgz
!cd LKH-3.0.7; make; cp LKH ..

LKH-3.0.7/
LKH-3.0.7/pr2392.par
LKH-3.0.7/whizzkids96.atsp
LKH-3.0.7/Makefile
LKH-3.0.7/whizzkids96.par
LKH-3.0.7/pr2392.tsp
LKH-3.0.7/DOC/
LKH-3.0.7/README.txt
LKH-3.0.7/SRC/
LKH-3.0.7/SRC/Penalty_CVRPTW.c
LKH-3.0.7/SRC/RestoreTour.c
LKH-3.0.7/SRC/SolveKMeansSubproblems.c
LKH-3.0.7/SRC/IsCommonEdge.c
LKH-3.0.7/SRC/Penalty_TSPPD.c
LKH-3.0.7/SRC/ReadProblem.c
LKH-3.0.7/SRC/BestKOptMove.c
LKH-3.0.7/SRC/Distance_SPECIAL.c
LKH-3.0.7/SRC/Penalty_TSPDL.c
LKH-3.0.7/SRC/Penalty_PDPTW.c
LKH-3.0.7/SRC/Penalty_ACVRP.c
LKH-3.0.7/SRC/CreateCandidateSet.c
LKH-3.0.7/SRC/OBJ/
LKH-3.0.7/SRC/Forbidden.c
LKH-3.0.7/SRC/Penalty_CCVRP.c
LKH-3.0.7/SRC/Penalty_M_PDTSP.c
LKH-3.0.7/SRC/Best5OptMove.c
LKH-3.0.7/SRC/RecordBetterTour.c
LKH-3.0.7/SRC/Best4OptMove.c
LKH-3.0.7/SRC/Exclude.c
LKH-3.0.7/SRC/C.c
LKH-3.0.7/SRC/IsCandidate.c
LKH-3.0.7/SRC/Make3OptMove.c
LKH-3.0.7/SRC/Make2OptMove.c
LKH-3.0.7/SRC/ResetCandidateSet.c
LKH-3.0.7/SRC/LKHmain.c
LKH-3.0.7/SRC/SolveSFCSubproblems.c
LKH-3.0.7/SRC/ERXT.c
LKH-3.0.7/S

make[2]: Leaving directory '/home/seffenberger/Documents/Private/Kaggle/Santa 2021/LKH-3.0.7/SRC'
make[1]: Leaving directory '/home/seffenberger/Documents/Private/Kaggle/Santa 2021/LKH-3.0.7/SRC'


In [26]:
LETTERS = {
    5: '🎅',  # father christmas
    4: '🤶',  # mother christmas
    1: '🦌',  # reindeer
    2: '🧝',  # elf
    3: '🎄',  # christmas tree
    6: '🎁',  # gift
    7: '🎀',  # ribbon
    8: '🌟',  # star
}
INV_LETTERS = {v: k for k, v in LETTERS.items()}

solution = pd.read_csv('submission_tsp1_wildcard_opt.csv')
strings = [[INV_LETTERS[c] for c in s] for s in solution.schedule]
strings.sort(key=len, reverse=True)
print(f'Strings lengths are {[len(_) for _ in strings]}.')

Strings lengths are [2501, 2498, 2496].


In [27]:
INV_LETTERS

{'🎅': 5, '🤶': 4, '🦌': 1, '🧝': 2, '🎄': 3, '🎁': 6, '🎀': 7, '🌟': 8}

In [28]:
def find_strings_perms(strings, verbose=False):
    all_perms = set(itertools.permutations(range(1, 8), 7))
    perms = []
    for s in strings:
        perms.append([])
        for i in range(len(s)-6):
            p = tuple(s[i:i+7])
            if p in all_perms:
                perms[-1].append(p)
    if verbose:
        lens = [len(_) for _ in  perms]
        print(f'There are {lens} permutations in strings, {sum(lens)} in total.')
        lens = [len(set(_)) for _ in  perms]
        print(f'There are {lens} unique permutations in strings, {sum(lens)} in total.')
    return perms

strings_perms = find_strings_perms(strings, verbose=True)

There are [1891, 1947, 1893] permutations in strings, 5731 in total.
There are [1869, 1932, 1862] unique permutations in strings, 5663 in total.


In [29]:
def rebalance_perms(strings_perms, verbose=False):
    # convert to dicts for fast lookup and to keep permutations order
    strings_perms = [dict.fromkeys(_) for _ in strings_perms] 
    for p in strings_perms[0].copy():  # iterate over the copy to allow modification during iteration
        if p[:2] != (1, 2) and (p in strings_perms[1] or p in strings_perms[2]):
            strings_perms[0].pop(p)
    for p in strings_perms[1].copy():
        if p[:2] != (1, 2) and p in strings_perms[2]:
            strings_perms[1].pop(p)
    if verbose:
        lens = [len(_) for _ in  strings_perms]
        print(f'There are {lens} permutations left in strings after rebalancing, {sum(lens)} in total.')
    return [list(_) for _ in strings_perms] 

strings_perms = rebalance_perms(strings_perms, verbose=True)

There are [1539, 1640, 1862] permutations left in strings after rebalancing, 5041 in total.


In [30]:
def perm_dist(p, q):
    i = p.index(q[0])
    return i if p[i:] == q[:7-i] else 7

def perms_to_string(perms):
    perms = list(perms)
    s = [*perms[0]]
    for p, q in zip(perms, perms[1:]):
        d = perm_dist(p, q)
        s.extend(q[-d:])
    return s

In [31]:
def distances_matrix(perms):
    m = np.zeros((len(perms), len(perms)), dtype='int8')
    for i, p in enumerate(perms):
        for j, q in enumerate(perms):
            m[i, j] = perm_dist(p, q)
    return m

def write_params_file():
    with open('santa.par', 'w') as f:
        print('PROBLEM_FILE = santa.atsp', file=f)
        print('TOUR_FILE = best_tour.txt', file=f)
        print('INITIAL_TOUR_FILE = initial_tour.txt', file=f)
        print('PATCHING_C = 4', file=f)
        print('PATCHING_A = 3', file=f)
        print('GAIN23 = YES', file=f)
        print('SEED = 42', file=f)
        print('MAX_TRIALS = 100000', file=f)
        print('TIME_LIMIT = 300', file=f) #seconds
        print('TRACE_LEVEL = 1', file=f)

def write_problem_file(distances):
    with open('santa.atsp', 'w') as f:
        print('TYPE: ATSP', file=f)
        print(f'DIMENSION: {len(distances)}', file=f)
        print('EDGE_WEIGHT_TYPE: EXPLICIT', file=f)
        print('EDGE_WEIGHT_FORMAT: FULL_MATRIX\n', file=f)
        print('EDGE_WEIGHT_SECTION', file=f)
        for row in distances:
            print(' '.join(str(_) for _ in row), file=f)

def write_initial_tour_file(perms):
    with open('initial_tour.txt', 'w') as f:
        print('TOUR_SECTION', file=f)
        print(' '.join(str(_) for _ in range(1, len(perms)+1)), -1, file=f)

def read_output_tour(perms):
    perms = list(perms)
    with open('best_tour.txt') as f:
        lines = f.readlines()
    tour = lines[lines.index('TOUR_SECTION\n')+1:-2]
    return [perms[int(_) - 1] for _ in tour] 
    
def solve_atsp(perms, verbose=False):
    write_params_file()
    distances = distances_matrix(perms)
    write_problem_file(distances)
    write_initial_tour_file(perms)
    
    # Run LKH-3 to solve ATSP instance
    if verbose:
        !./LKH santa.par
    else:
        !touch lkh.log
        !./LKH santa.par >> lkh.log
    tour = read_output_tour(perms)
    return perms_to_string(tour)

In [32]:
improved, old_lens = True, [len(_) for _ in strings]
while improved:
    print('='*91)
    new_strings = [solve_atsp(_) for _ in strings_perms]
    new_strings.sort(key=len, reverse=True)
    new_lens = [len(_) for _ in new_strings]
    if new_lens < old_lens:
        print(f'Improved strings lengths from {old_lens} to {new_lens}.')
        strings, old_lens = new_strings, new_lens
        strings_perms = find_strings_perms(strings, verbose=True)
        strings_perms = rebalance_perms(strings_perms, verbose=True)
    else:
        improved = False

Improved strings lengths from [2501, 2498, 2496] to [2488, 2106, 1947].
There are [1908, 1711, 1597] permutations in strings, 5216 in total.
There are [1887, 1703, 1594] unique permutations in strings, 5184 in total.
There are [1770, 1681, 1594] permutations left in strings after rebalancing, 5045 in total.
Improved strings lengths from [2488, 2106, 1947] to [2230, 2079, 1947].
There are [1801, 1703, 1597] permutations in strings, 5101 in total.
There are [1794, 1695, 1594] unique permutations in strings, 5083 in total.
There are [1762, 1689, 1594] permutations left in strings after rebalancing, 5045 in total.
Improved strings lengths from [2230, 2079, 1947] to [2219, 2079, 1947].
There are [1790, 1703, 1597] permutations in strings, 5090 in total.
There are [1786, 1695, 1594] unique permutations in strings, 5075 in total.
There are [1762, 1689, 1594] permutations left in strings after rebalancing, 5045 in total.
Improved strings lengths from [2219, 2079, 1947] to [2218, 2079, 1947].
T

In [33]:
all_perms = set(itertools.permutations(range(1, 8), 7))
mandatory_perms = set((1, 2) +  _ for _ in itertools.permutations(range(3, 8), 5))

strings_perms = [set(_) for _ in find_strings_perms(strings)]
for i, s in enumerate(strings_perms):
    if mandatory_perms - s:
        print(f'String #{i} is missing {mandatory_perms - s}.')
if all_perms - set.union(*strings_perms):
    print(f'Strings are missing {all_perms - set.union(*strings_perms)}.')

String #0 is missing {(1, 2, 3, 4, 5, 6, 7), (1, 2, 5, 6, 4, 7, 3), (1, 2, 5, 7, 6, 3, 4), (1, 2, 3, 6, 5, 7, 4), (1, 2, 6, 7, 4, 5, 3), (1, 2, 5, 6, 3, 4, 7), (1, 2, 6, 5, 4, 3, 7), (1, 2, 3, 6, 4, 7, 5), (1, 2, 5, 6, 7, 4, 3), (1, 2, 7, 4, 3, 5, 6), (1, 2, 7, 4, 3, 6, 5), (1, 2, 6, 3, 5, 7, 4), (1, 2, 5, 4, 3, 7, 6), (1, 2, 7, 3, 6, 4, 5), (1, 2, 4, 3, 5, 7, 6), (1, 2, 6, 7, 4, 3, 5), (1, 2, 6, 4, 5, 3, 7), (1, 2, 4, 3, 7, 5, 6), (1, 2, 3, 7, 6, 4, 5), (1, 2, 6, 4, 3, 7, 5), (1, 2, 5, 3, 6, 7, 4), (1, 2, 5, 4, 7, 6, 3), (1, 2, 5, 3, 4, 6, 7), (1, 2, 3, 6, 7, 4, 5), (1, 2, 4, 6, 5, 7, 3), (1, 2, 6, 5, 3, 7, 4), (1, 2, 6, 4, 3, 5, 7), (1, 2, 5, 4, 7, 3, 6), (1, 2, 7, 3, 4, 5, 6), (1, 2, 6, 3, 7, 4, 5), (1, 2, 6, 5, 3, 4, 7), (1, 2, 4, 7, 3, 6, 5), (1, 2, 3, 5, 4, 7, 6), (1, 2, 6, 5, 7, 4, 3), (1, 2, 4, 6, 3, 5, 7), (1, 2, 4, 7, 3, 5, 6), (1, 2, 5, 6, 7, 3, 4), (1, 2, 3, 5, 4, 6, 7), (1, 2, 4, 3, 6, 7, 5), (1, 2, 7, 4, 5, 3, 6), (1, 2, 4, 3, 7, 6, 5), (1, 2, 3, 6, 4, 5, 7), (1, 2, 5, 6,

In [34]:
sub = pd.DataFrame()
sub['schedule'] = [''.join(LETTERS[x] for x in s) for s in strings]
sub_name = f'submission_tsp_6_perm_rebalance_no_wildcards_{"_".join(str(len(_)) for _ in strings)}.csv'
sub.to_csv(sub_name, index=False)